In [2]:
import os
import hashlib
import re

In [3]:
DATASET_DIR = './Dataset/'

In [4]:
classCounts = {"train":{},"test":{}}
dirPath = os.path.join(DATASET_DIR,'test')
for classFolder in os.listdir(dirPath):
    classCounts['test'][classFolder] = len(os.listdir(os.path.join(dirPath, classFolder)))

dirPath = os.path.join(DATASET_DIR,'train/audio')
for classFolder in os.listdir(dirPath):
    classCounts['train'][classFolder] = len(os.listdir(os.path.join(dirPath, classFolder)))


for splitName, splitObject in classCounts.items():
    print(f"Class distribution for {splitName}")
    for subfolder, count in splitObject.items():
        print(f"{subfolder}: {count}")

Class distribution for train
bed: 1713
bird: 1731
cat: 1733
dog: 1746
down: 2359
eight: 2352
five: 2357
four: 2372
go: 2372
happy: 1742
house: 1750
left: 2353
marvin: 1746
nine: 2364
no: 2375
off: 2357
on: 2367
one: 2370
right: 2367
seven: 2377
sheila: 1734
six: 2369
stop: 2380
three: 2356
tree: 1733
two: 2373
up: 2375
wow: 1745
yes: 2377
zero: 2376
_background_noise_: 7
Class distribution for test
audio: 158538


## Creating train, valid, test split from the train folder

In [28]:
MAX_NUM_WAVS_PER_CLASS = 2**27 - 1  # ~134M

def which_set(filename, validation_percentage, testing_percentage):
  """Determines which data partition the file should belong to.

  We want to keep files in the same training, validation, or testing sets even
  if new ones are added over time. This makes it less likely that testing
  samples will accidentally be reused in training when long runs are restarted
  for example. To keep this stability, a hash of the filename is taken and used
  to determine which set it should belong to. This determination only depends on
  the name and the set proportions, so it won't change as other files are added.

  It's also useful to associate particular files as related (for example words
  spoken by the same person), so anything after '_nohash_' in a filename is
  ignored for set determination. This ensures that 'bobby_nohash_0.wav' and
  'bobby_nohash_1.wav' are always in the same set, for example.

  Args:
    filename: File path of the data sample.
    validation_percentage: How much of the data set to use for validation.
    testing_percentage: How much of the data set to use for testing.

  Returns:
    String, one of 'training', 'validation', or 'testing'.
  """
  base_name = os.path.basename(filename)
  # We want to ignore anything after '_nohash_' in the file name when
  # deciding which set to put a wav in, so the data set creator has a way of
  # grouping wavs that are close variations of each other.
  hash_name = re.sub(r'_nohash_.*$', '', base_name).encode()
  # This looks a bit magical, but we need to decide whether this file should
  # go into the training, testing, or validation sets, and we want to keep
  # existing files in the same set even if more files are subsequently
  # added.
  # To do that, we need a stable way of deciding based on just the file name
  # itself, so we do a hash of that and then use that to generate a
  # probability value that we use to assign it.
  hash_name_hashed = hashlib.sha1(hash_name).hexdigest()
  percentage_hash = ((int(hash_name_hashed, 16) %
                      (MAX_NUM_WAVS_PER_CLASS + 1)) *
                     (100.0 / MAX_NUM_WAVS_PER_CLASS))
  if percentage_hash < validation_percentage:
    result = 'validation'
  elif percentage_hash < (testing_percentage + validation_percentage):
    result = 'testing'
  else:
    result = 'training'
  return result


In [36]:
def getFileClass(filePath):
    dir = os.path.dirname(filePath)
    return dir.split('\\')[-1]

In [30]:
valid_per= 10
test_per =10
valid_fileNames=[]
test_fileNames=[]
train_fileNames=[]

for subdir, dirs, files in os.walk('./Dataset/train/audio'):
    for file in files:
        # directory = subdir.split('\\')[-1]
        match(which_set(file,valid_per,test_per)):
            case 'training':
                train_fileNames.append(os.path.join(subdir,file))
            case 'validation':
                valid_fileNames.append(os.path.join(subdir,file))
            case 'testing':
                test_fileNames.append(os.path.join(subdir,file))

In [1]:
getFileClass(valid_fileNames[0])

NameError: name 'getFileClass' is not defined

## We can handle the files the way we want, by either loading them all to memory or creating a stream thingy